# Image registration for Sequioa Cherry

As it is a sequoia orthomosaic, scaled from 0-to-1, we assume it is calibrated
* For the following Dataset: https://zenodo.org/records/7144071 
* Using code snippets from:

Output: `uint16` 0-65535 scaled .tif files with green, red, rededge, nir bands


In [2]:
import os
import rioxarray as rxr
import xarray as xr
import tifffile
from glob import glob
import numpy as np
from itertools import repeat

In [3]:
def load_image(path):
    return rxr.open_rasterio(path), path
    
def rescale_to_uint16(da):
    da = da.where(~np.isnan(da), 65535)
    rescaled = (da * 65535).clip(0, 65535).astype(np.uint16)
    
    return rescaled

def rescale_float32_to_uint16_xarray(xarr):
    # Mask NaNs and -1000 values
    xarr = xr.where(xarr == -10000.0, 1, xarr)
    
    # Find min and max values ignoring NaNs
    min_val = 0
    max_val = 1
    
    # Normalize to [0, 1] ignoring NaNs
    normalized = (xarr - min_val) / (max_val - min_val)
    
    # Scale to [0, 65534]
    scaled = (normalized * 65535).astype(np.uint16)
    
    # Set NaNs and -1000 to 65535
    scaled = scaled.where(~xarr.isnull() & (xarr != -1000), 65535)
    
    return scaled
    
def create_4_channel_image(green, red,rededge, nir):
    """Create a 4-channel image (G, R, RE, NIR)."""
    return np.stack((green, red, rededge, nir), axis=-1)
    

def save_image(filename, image, metadata=None):
    """Save the image as a 4-channel TIFF using tifffile."""
    #tifffile.imwrite(filename, image, photometric='rgb',extratags=metadata[0],subifds = metadata[1])
    if metadata is not None:
        extratags = metadata[0]
        subifds = metadata[1]
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb',
                extratags=extratags,
                subifds=subifds
            )
    else:
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb')
            


def process_multispec_set(green_image_path, output_directory):

    base_name = os.path.basename(green_image_path).replace("green.data.tif", "")
    # Load the green band image (reference image)
    dirname = os.path.dirname(green_image_path)
    
    green,_ = load_image(green_image_path)
    red,_ = load_image(os.path.join(dirname, base_name+"red.data.tif"))
    rededge,_ = load_image(os.path.join(dirname, base_name+"rededge.data.tif"))
    nir,_ = load_image(os.path.join(dirname, base_name+"nir.data.tif"))
    green = rescale_float32_to_uint16_xarray(green)
    red = rescale_float32_to_uint16_xarray(red)
    rededge = rescale_float32_to_uint16_xarray(rededge)
    nir = rescale_float32_to_uint16_xarray(nir)

    
     # Create a 4-channel image
    image_4ch = create_4_channel_image(green, red, rededge, nir)
    
    
    if len(image_4ch.shape)>3:
        image_4ch = np.squeeze(image_4ch)

    sensor = "SEQUOIA"
    cal = "CAL"
    set_title = os.path.basename(os.path.normpath(output_directory))
    output_filename = f"{sensor}_{cal}_{set_title}_{base_name}.tif"
    output_path = os.path.join(output_directory, output_filename)
    os.makedirs(output_directory, exist_ok=True)
    

    # Save the 4-channel image
    save_image(output_path, image_4ch)

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def process_directory(input_dir, output_directory, n_threads= 1):
    """Process an entire directory with multiple sets of multispectral images."""
    input_dir = os.path.abspath(input_dir)
    os.makedirs(output_directory, exist_ok=True)

    # Find all green band images (reference images)
    green_images = glob(os.path.join(input_dir, "*green*.data.tif"))

    # Process sets of 4 multispectral images in parallel with a progress bar
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        list(tqdm(executor.map(process_multispec_set, green_images, repeat(output_directory)), total=len(green_images)))

    print("All images successfully processed.")

In [4]:
folder = "../../data/MS_pretraining/Sequoia_cherry/03-11-2021/"
input_folder = folder
output_folder = "../../data/processed/sequioa_cherry_a/"
process_directory(input_folder, output_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.87it/s]

All images successfully processed.


In [5]:
folder = "../../data/MS_pretraining/Sequoia_cherry/08-07-2021/"
input_folder = folder
output_folder = "../../data/processed/sequioa_cherry_b/"
process_directory(input_folder, output_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.06it/s]

All images successfully processed.


In [6]:
folder = "../../data/MS_pretraining/Sequoia_cherry/13-07-2022/"
input_folder = folder
output_folder = "../../data/processed/sequioa_cherry_c/"
process_directory(input_folder, output_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.92it/s]

All images successfully processed.


In [7]:
folder = "../../data/MS_pretraining/Sequoia_cherry/16-09-2021/"
input_folder = folder
output_folder = "../../data/processed/sequioa_cherry_d/"
process_directory(input_folder, output_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.99it/s]

All images successfully processed.


In [8]:
folder = "../../data/MS_pretraining/Sequoia_cherry/26-05-2022/"
input_folder = folder
output_folder = "../../data/processed/sequioa_cherry_e/"
process_directory(input_folder, output_folder)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.86it/s]

All images successfully processed.
